API - Datos Red Eléctrica

In [9]:
import requests
#import pandas as pd
import dotenv as dotenv
import os


# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os
import pandas as pd
import re

In [10]:
cod_comunidades = {'Ceuta': 8744,
                        'Melilla': 8745,
                        'Andalucía': 4,
                        'Aragón': 5,
                        'Cantabria': 6,
                        'Castilla - La Mancha': 7,
                        'Castilla y León': 8,
                        'Cataluña': 9,
                        'País Vasco': 10,
                        'Principado de Asturias': 11,
                        'Comunidad de Madrid': 13,
                        'Comunidad Foral de Navarra': 14,
                        'Comunitat Valenciana': 15,
                        'Extremadura': 16,
                        'Galicia': 17,
                        'Illes Balears': 8743,
                        'Canarias': 8742,
                        'Región de Murcia': 21,
                        'La Rioja': 20}

In [11]:
anios = [2019,2020,2021]
# me hago la lista de años que busco

API - Demanda Eléctrica

In [91]:
headers={"Accept": "application/json",
"Content-Type": "application/json",
"Host": "apidatos.ree.es"}
# creo mi destino
ruta = "C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos_demanda_electrica"
os.makedirs(ruta, exist_ok=True)
# si existe bien, si no pues la crea
for i in anios: #itero en años
    for name, cod in cod_comunidades.items(): #itero en comunicades
        url_demanda_electrica = f'https://apidatos.ree.es/es/datos/demanda/evolucion?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=month&geo_trunk=electric_system&geo_limit=ccaa&geo_ids={cod}'

        response = requests.get(url_demanda_electrica, headers=headers)
        if response.status_code == 200:
            response_json= response.json()
            df_demanda_electrica= pd.DataFrame(response_json['included'][0]['attributes']['values']) #tomo el primero porque solo hay uno
            df_demanda_electrica['COD-CCAA'] = cod
            df_demanda_electrica.to_csv(os.path.join(ruta, f'{name}_{i}.csv'))
        else: 
            print(f'El error es {response.status_code} en {name} para el annio {i}')

In [72]:
response_json

{'data': {'type': 'Evolución de la demanda',
  'id': 'dem1',
  'attributes': {'title': 'Evolución de la demanda',
   'last-update': '2022-01-11T09:21:30.000+01:00',
   'description': None},
  'meta': {'cache-control': {'cache': 'HIT',
    'expireAt': '2024-11-28T05:55:18'}}},
 'included': [{'type': 'Demanda',
   'id': '10339',
   'groupId': None,
   'attributes': {'title': 'Demanda',
    'description': None,
    'color': '#ffea00',
    'type': None,
    'magnitude': None,
    'composite': False,
    'last-update': '2022-11-10T23:30:07.000+01:00',
    'values': [{'value': 146298.76,
      'percentage': 1,
      'datetime': '2021-01-01T00:00:00.000+01:00'},
     {'value': 129503.338,
      'percentage': 1,
      'datetime': '2021-02-01T00:00:00.000+01:00'},
     {'value': 144652.408,
      'percentage': 1,
      'datetime': '2021-03-01T00:00:00.000+01:00'},
     {'value': 129506.033,
      'percentage': 1,
      'datetime': '2021-04-01T00:00:00.000+02:00'},
     {'value': 130651.022,
   

In [73]:
response_json['included'][0]

{'type': 'Demanda',
 'id': '10339',
 'groupId': None,
 'attributes': {'title': 'Demanda',
  'description': None,
  'color': '#ffea00',
  'type': None,
  'magnitude': None,
  'composite': False,
  'last-update': '2022-11-10T23:30:07.000+01:00',
  'values': [{'value': 146298.76,
    'percentage': 1,
    'datetime': '2021-01-01T00:00:00.000+01:00'},
   {'value': 129503.338,
    'percentage': 1,
    'datetime': '2021-02-01T00:00:00.000+01:00'},
   {'value': 144652.408,
    'percentage': 1,
    'datetime': '2021-03-01T00:00:00.000+01:00'},
   {'value': 129506.033,
    'percentage': 1,
    'datetime': '2021-04-01T00:00:00.000+02:00'},
   {'value': 130651.022,
    'percentage': 1,
    'datetime': '2021-05-01T00:00:00.000+02:00'},
   {'value': 128050.949,
    'percentage': 1,
    'datetime': '2021-06-01T00:00:00.000+02:00'},
   {'value': 140864.511,
    'percentage': 1,
    'datetime': '2021-07-01T00:00:00.000+02:00'},
   {'value': 130850.035,
    'percentage': 1,
    'datetime': '2021-08-01T0

In [62]:
len(response_json['included'])

1

Api - Generación

In [89]:
headers={"Accept": "application/json",
"Content-Type": "application/json",
"Host": "apidatos.ree.es"}

ruta_g = "C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos_generacion_estructura"
os.makedirs(ruta_g, exist_ok=True)

for i in anios:
    for name, cod in cod_comunidades.items():
        url_generacion_renovable = f'https://apidatos.ree.es/es/datos/generacion/estructura-renovables?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=month&geo_trunk=electric_system&geo_limit=ccaa&geo_ids={cod}'

        response_generacion = requests.get(url_generacion_renovable, headers=headers) # me dio problemas, veamos el porqué
        if response_generacion.status_code == 200:
            response_generacion_json = response_generacion.json()
            lista_df = []
            for j in range(len(response_generacion_json['included'])): #itero aquí porque son varios elementos 
                df_generacion_estructura = pd.DataFrame(response_generacion_json['included'][j]['attributes']['values'])
                df_generacion_estructura['type'] = response_generacion_json['included'][j]['type']
                df_generacion_estructura['COD-CCAA'] = cod
                lista_df.append(df_generacion_estructura)
            df_final = pd.concat(lista_df).to_csv(os.path.join(ruta_g, f'{name}_{i}.csv'))
        else:
            print(f"Error {response_generacion.status_code} en la {name} para el año {i}")

        #recuerda que te equivocaste primero, prueba con response_json
        # será que se sobreescribe o se apendea
        # trata de poner el codigo de cada comunidad

Error 502 en la Ceuta para el año 2019
Error 502 en la Ceuta para el año 2020
Error 503 en la Cataluña para el año 2020
Error 502 en la Ceuta para el año 2021


In [47]:
response_generacion_json.keys()

dict_keys(['data', 'included'])

In [88]:
response_generacion_json['included']

[{'type': 'Hidráulica',
  'id': '10330',
  'groupId': '1',
  'attributes': {'title': 'Hidráulica',
   'description': 'Hidráulica: no incluye la generación bombeo.',
   'color': '#0090d1',
   'type': None,
   'magnitude': None,
   'composite': False,
   'last-update': '2022-01-13T12:43:08.000+01:00',
   'values': [{'value': 15402.108,
     'percentage': 0.1042379755922421,
     'datetime': '2021-01-01T00:00:00.000+01:00'},
    {'value': 20891.095,
     'percentage': 0.19116304592730085,
     'datetime': '2021-02-01T00:00:00.000+01:00'},
    {'value': 17755.933,
     'percentage': 0.1289102793122252,
     'datetime': '2021-03-01T00:00:00.000+01:00'},
    {'value': 12484.9,
     'percentage': 0.11947567243380085,
     'datetime': '2021-04-01T00:00:00.000+02:00'},
    {'value': 12626.526,
     'percentage': 0.18238770633844256,
     'datetime': '2021-05-01T00:00:00.000+02:00'},
    {'value': 10694.393,
     'percentage': 0.15929420573501224,
     'datetime': '2021-06-01T00:00:00.000+02:00'

In [63]:
len(response_generacion_json['included'])

5

In [78]:
len(response_generacion_json['included'][1]['attributes']['values'])

12

In [85]:
response_generacion_json['included'][1]

{'type': 'Eólica',
 'id': '10333',
 'groupId': '1',
 'attributes': {'title': 'Eólica',
  'description': None,
  'color': '#6fb114',
  'type': None,
  'magnitude': None,
  'composite': False,
  'last-update': '2022-11-10T23:30:06.000+01:00',
  'values': [{'value': 126564.559,
    'percentage': 0.8565602456420176,
    'datetime': '2021-01-01T00:00:00.000+01:00'},
   {'value': 80023.713,
    'percentage': 0.7322534660577698,
    'datetime': '2021-02-01T00:00:00.000+01:00'},
   {'value': 106078.27,
    'percentage': 0.7701414177817432,
    'datetime': '2021-03-01T00:00:00.000+01:00'},
   {'value': 78024.301,
    'percentage': 0.7466624344730259,
    'datetime': '2021-04-01T00:00:00.000+02:00'},
   {'value': 39047.666,
    'percentage': 0.564035922438966,
    'datetime': '2021-05-01T00:00:00.000+02:00'},
   {'value': 38798.185,
    'percentage': 0.5779033988684599,
    'datetime': '2021-06-01T00:00:00.000+02:00'},
   {'value': 55639.61,
    'percentage': 0.635935918560497,
    'datetime': '